# Notebook for making predictions with an ensemble of SuperLearner machine learning models.

This Jupyter notebook is an example for how to reuse an archived ML model trained for predicting sediment respiration rates. The general concepts used here can be applied elsewhere. The core operations used here are based on `sl_core/predict.py` available at [this link](https://github.com/parallelworks/sl_core/blob/main/predict.py).

This notebook proceeds in two **independent** stages. The first set of predictions are a relatively large job over some 80,000 sites (10 mins run time, 46GB RAM) - these are all the possible RiverAtlas sites in a whole river basin.  The second stage is predictions at the RiverAtlas and GLORICH merged sites (so limited to the number of sites where there are records of _in situ_ chemistry observations) instead of all river segments identified in topography.

## Dependencies

In order to use the ML model, you need to first get access to the Python packages necessary for running the model. SuperLearners are currently stored in `.pkl` format and this format is sensitive to the exact versions of Python and scikit-learn that are in the active environment. (Future work will move SuperLearners to ONNX format which is more portable.) The SuperLearner automatically stores `.yaml` files that define its run environment, but, to keep environments lightweight and minimize install time, etc., these environments do not contain the packages needed for displaying Jupyter notebooks. As such, this repository contains a `.yaml` that can be used with the following command:
```
conda env update --name <your-env-name> -f fig07-08-notebook-conda-env.yaml
```

This file was created from an automatically generated SuperLearner environment definition file with the following commands:
```
conda create -y --name superlearner python=3.9
conda activate superlearner
conda env update --name superlearner -f requirements.yaml
conda install -y -c conda-forge requests
conda install -y -c anaconda jinja2
conda install -y -c conda-forge ipykernel
conda env export --name superlearner > fig07-08-notebook-conda-env.yaml
```

The `.yaml` files here are stored as gzipped files `.yaml.gz` because otherwise GitHub will read the `requirements.yaml` files and print security warnings if the files use out of date packages. Since this code is of limited duration scientific use (i.e. not production) I have ignored these warnings.

In [1]:
import pickle
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
import sys
import o2sat

## Stage 1 (RA only): Specify repository and branch to work on

In [2]:
# Using ~ here for $HOME will cause pickle.load failures later
# so must use absolute path.
repo_prefix = '/home/sfgary/tmp/'
repo_name = 'sl-archive-whondrs'
repo_url = 'https://github.com/parallelworks/'+repo_name
branch = 'S19S-SSS-log10-extrap-r02'

# Grab the data and get onto the branch if not already there
! mkdir -p {repo_prefix}
! cd {repo_prefix}; git clone {repo_url}
! cd {repo_prefix}/{repo_name}; git checkout {branch}

fatal: destination path 'sl-archive-whondrs' already exists and is not an empty directory.
Switched to branch 'S19S-SSS-log10-extrap-r02'


## Stage 1 (RA only): Load data that will be used to make predictions

In [3]:
# There are two zipped files.  Decompress and load each one
!gunzip -c grdb_step_03b_output_RiverATLAS_v10_na.xyz.7411.csv.gz > grdb_step_03b_output_RiverATLAS_v10_na.xyz.7411.csv
!gunzip -c grdb_step_03b_output_RiverATLAS_v10_na.xyz.7412.csv.gz > grdb_step_03b_output_RiverATLAS_v10_na.xyz.7412.csv

# Do not specify ID as the index when loading the data because
# we will want to remove it later.
list_df = []
list_df.append(pd.read_csv('grdb_step_03b_output_RiverATLAS_v10_na.xyz.7411.csv')) #, index_col='RA_ID'))
list_df.append(pd.read_csv('grdb_step_03b_output_RiverATLAS_v10_na.xyz.7412.csv')) #, index_col='RA_ID'))

# Clean up
!rm grdb_step_03b_output_RiverATLAS_v10_na.xyz.7411.csv
!rm grdb_step_03b_output_RiverATLAS_v10_na.xyz.7412.csv

# Concatenate
predict_df = pd.concat(list_df,axis=0)

# Some river segments are so short they only have one coordinate point.
# Replace any missing (lon2,lat2) with (lon1,lat2) for uniformity.
# Add a very very small displacement to each so all segments have
# very small but non-zero length.
predict_df['lon2'].fillna(value=predict_df['lon1']+0.000001,inplace=True)
predict_df['lat2'].fillna(value=predict_df['lat1']+0.000001,inplace=True)

# Store the ID, lon, and lat in a separate dataframe for integration later
# These values are NOT used by the ML model and and such should be removed
# from the DF that is going to be used to make predictions.
predict_ixy = pd.DataFrame(columns=['RA_ID','lon1','lat1','lon2','lat2'])
predict_ixy['RA_ID'] = predict_df.pop('RA_ID')
predict_ixy['lon1'] = predict_df.pop('lon1')
predict_ixy['lat1'] = predict_df.pop('lat1')
predict_ixy['lon2'] = predict_df.pop('lon2')
predict_ixy['lat2'] = predict_df.pop('lat2')

# Reset the index to increment to column order - this will match the
# output from the predict stage later.
predict_ixy.reset_index(drop=True,inplace=True)

# There is exactly one NaN value remaining at one site for the stream
# depth. For generality with other data sets, replace any NaN
# with the mean value of the whole column.
predict_df.fillna(predict_df.mean(), inplace = True)

## Stage 1 (RA only): Make predictions

In [4]:
# Set number of SuperLearner ensemble members
num_sl = 10

# Initialize data frame list to hold output
sl_predict_output_df_list = []

# Loop over all ensemble members
for ll in range(0,num_sl):
    
    print("Working on SuperLearner ensemble member "+str(ll))
    
    # Load the SuperLearner model from .pkl
    model_dir = repo_prefix+repo_name+"/ml_models/sl_"+str(ll)
    sys.path.append(model_dir)
    with open(model_dir+'/SuperLearners.pkl','rb') as file_object:
        superlearner = pickle.load(file_object)

    # OPTIONAL: For a given output variable, list the models:
    predict_var = 'Normalized_Respiration_Rate_mg_DO_per_H_per_L_sediment'
    #print("Submodels within SuperLearner and their weights:")
    #list_models = list(superlearner[predict_var].named_estimators_.keys())
    #print(list_models)
    
    # OPTIONAL: The following only works for the scipy.optimize.nnls
    # stacking regressor, not the sklearn stacking regressors.
    #print(superlearner[predict_var].final_estimator_.weights_)
    
    # Make predictions
    sl_predict_output_df_list.append(superlearner[predict_var].predict(predict_df))

Working on SuperLearner ensemble member 0


/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/en

Working on SuperLearner ensemble member 1


/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/en

Working on SuperLearner ensemble member 2


...

/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/en

Working on SuperLearner ensemble member 8


/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/en

Working on SuperLearner ensemble member 9


/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/en

# Stage 1 (RA only): Consolidate over ensemble members

In [5]:
# Didn't realize that .predict output is a pure numpy array and not a DF.
# I don't want to rerun the cell above (10 mins) so consolidate to DF here.
sl_predict_output = pd.DataFrame(columns=['sl_0','sl_1','sl_2','sl_3','sl_4','sl_5','sl_6','sl_7','sl_8','sl_9'])

# Loop over all ensemble members
for ll in range(0,num_sl):
    sl_predict_output['sl_'+str(ll)] = sl_predict_output_df_list[ll]

# Grab mean and std
sl_predict_output['mean'] = sl_predict_output.mean(axis=1)
sl_predict_output['std'] = sl_predict_output.std(axis=1)

# Paste the ixy info back into the file so we can plot the network of rivers
# with GMT (not this notebook). Reset index to row order
out = pd.concat([predict_ixy, sl_predict_output], axis='columns')
out.to_csv('grdb_'+branch+'_predictions.csv')

# Switch to Stage 2: Predict over the RiverAtlas+GLORICH merged sites

## Stage 2 (RA+GH): Specify repository and branch to work on

In [24]:
# Using ~ here for $HOME will cause pickle.load failures later
# so must use absolute path.
repo_prefix = '/home/sfgary/tmp/'
repo_name = 'sl-archive-whondrs'
repo_url = 'https://github.com/parallelworks/'+repo_name
branch = 'S19S-SSS-log10-extrap-r01a'

# Grab the data and get onto the branch if not already there
! mkdir -p {repo_prefix}
! cd {repo_prefix}; git clone {repo_url}
! cd {repo_prefix}/{repo_name}; git checkout {branch}

fatal: destination path 'sl-archive-whondrs' already exists and is not an empty directory.
Checking out files: 100% (274/274), done.
Switched to branch 'S19S-SSS-log10-extrap-r01a'


## Stage 2 (RA+GH): Load data that will be used to make predictions

In [25]:
# This is a smaller dataset and loads from a single file
# Data are at a single point (not a river segment with
# minimum of 2 points)
predict_df = pd.read_csv('RiverAtlas_GLORICH_colocated_for_prediction.csv')

# Using this line of code: np.sum(np.isnan(predict_df))
# there are no missing temperature values, 84 missing
# pH, 7534 missing DO, 4797 missing DO%sat and lots of
# missing TOC.  So, first off, we're just going to ignore TOC:
predict_df.pop('TOC')

# Also, these values were not used in the training of the
# ML model since they hold duplicate or irrelevant information
predict_df.pop('my_lm')
predict_df.pop('STAT_ID')
predict_df.pop('dist_m')

#============================================
# First attempt - reconstruct as much oxygen
# as possible
#============================================
if ( 1 == 1 ):
    # Attempt to reconstruct DO from temp and DO%sat.
    # Loop over all rows
    for index, row in predict_df.iterrows():

        # Must have temperature to attempt reconstruction
        if ( not np.isnan(row['Mean_Temp_Deg_C']) ):
            o2_sat_mg_per_l = o2sat.sw_o2sat(0.0, row['Mean_Temp_Deg_C'])*1.4291
        
            if (np.isnan(row['Mean_DO_mg_per_L']) and not np.isnan(row['Mean_DO_percent_saturation'])):
                # Compute any missing DO_mg_per_L from T and DOSAT  
                predict_df.at[index,'Mean_DO_mg_per_L'] = row['Mean_DO_percent_saturation']*o2_sat_mg_per_l/100.0
            elif (not np.isnan(row['Mean_DO_mg_per_L']) and np.isnan(row['Mean_DO_percent_saturation'])):
                # Compute any missing DOSAT from T and DO_mg_per_L.
                predict_df.at[index,'Mean_DO_percent_saturation'] = 100.0*row['Mean_DO_mg_per_L']/o2_sat_mg_per_l 

# Result: still  have 84 missing pH, and missing DO and DO%sat
# are both at 4350 points. So a substantial amount of points
# will have no DO information. Predictions with
# ML model have distinct strong respiration rate bias.

#================================================
# Second attempt - skip all missing values
#================================================
if ( 1 == 1 ):
    predict_df.dropna(axis='index', how='any', inplace=True)

# Result: only 2540 rows remain. Predictions with ML
# model have even stronger distinct respiration rate bais.

#================================================
# Third attempt - attempt to recover oxygen, above
# but also drop any missing values, also above.
# Then, shift the mean T, pH, DO, and DOSAT by
# the difference between the WHONDRS and GLORICH
# data sets so there is no mean bias. Biases from
# fig09_explore_WHONDRS_GLORICH_bias.ipynb
#================================================

# If set to 0, do the recovery and drop missing
# filters above, but leave the GLORICH insitu
# values unadjusted.

if ( 1 == 1 ):
    predict_df['Mean_Temp_Deg_C'] = predict_df['Mean_Temp_Deg_C'] + (2.08407431976158)
    predict_df['pH'] = predict_df['pH'] + (-0.18662680695643363)
    predict_df['Mean_DO_mg_per_L'] = predict_df['Mean_DO_mg_per_L'] + (-1.332851782847161)
    predict_df['Mean_DO_percent_saturation'] = predict_df['Mean_DO_percent_saturation'] + (-4.177509541392979)

# Result: 6170 rows remain. For S19S-SSS-log10-extrap-r01,
# this adjustment does not make a difference. So remove
# the adjustment when evaluating S19S-SSS-log10-extrap-r01a.

# For generality with other data sets, replace any NaN
# with the mean value of the whole column. 
predict_df.fillna(predict_df.mean(), inplace = True)

# Store the Sample_ID, Sample_Longitude, and Sample_Latitude 
# in a separate dataframe for integration later
# These values are NOT used by the ML model and and such should be removed
# from the DF that is going to be used to make predictions.
predict_ixy = pd.DataFrame(columns=['Sample_ID','Sample_Longitude','Sample_Latitude'])
predict_ixy['Sample_ID'] = predict_df.pop('Sample_ID')
predict_ixy['Sample_Longitude'] = predict_df.pop('Sample_Longitude')
predict_ixy['Sample_Latitude'] = predict_df.pop('Sample_Latitude')

# Reset the index to increment to column order - this will match the
# output from the predict stage later.
predict_ixy.reset_index(drop=True,inplace=True)

## Stage 2 (RA+GH): Make predictions

In [26]:
# Set number of SuperLearner ensemble members
num_sl = 10

# Initialize data frame list to hold output
sl_predict_output_df_list = []

# Loop over all ensemble members
for ll in range(0,num_sl):
    
    print("Working on SuperLearner ensemble member "+str(ll))
    
    # Load the SuperLearner model from .pkl
    model_dir = repo_prefix+repo_name+"/ml_models/sl_"+str(ll)
    sys.path.append(model_dir)
    with open(model_dir+'/SuperLearners.pkl','rb') as file_object:
        superlearner = pickle.load(file_object)

    # OPTIONAL: For a given output variable, list the models:
    predict_var = 'Normalized_Respiration_Rate_mg_DO_per_H_per_L_sediment'
    #print("Submodels within SuperLearner and their weights:")
    #list_models = list(superlearner[predict_var].named_estimators_.keys())
    #print(list_models)
    
    # OPTIONAL: The following only works for the scipy.optimize.nnls
    # stacking regressor, not the sklearn stacking regressors.
    #print(superlearner[predict_var].final_estimator_.weights_)
    
    # Make predictions
    sl_predict_output_df_list.append(superlearner[predict_var].predict(predict_df))

Working on SuperLearner ensemble member 0


/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/en

Working on SuperLearner ensemble member 1


/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/en

Working on SuperLearner ensemble member 2


...

/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/en

Working on SuperLearner ensemble member 8


/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/en

Working on SuperLearner ensemble member 9


/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/envs/superlearner/lib/python3.9/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/sfgary/.miniconda3c/en

# Stage 2 (RA+GH): Consolidate over ensemble members

In [27]:
# Didn't realize that .predict output is a pure numpy array and not a DF.
# I don't want to rerun the cell above (10 mins) so consolidate to DF here.
sl_predict_output = pd.DataFrame(columns=['sl_0','sl_1','sl_2','sl_3','sl_4','sl_5','sl_6','sl_7','sl_8','sl_9'])

# Loop over all ensemble members
for ll in range(0,num_sl):
    sl_predict_output['sl_'+str(ll)] = sl_predict_output_df_list[ll]

# Grab mean and std
sl_predict_output['mean'] = sl_predict_output.mean(axis=1)
sl_predict_output['std'] = sl_predict_output.std(axis=1)

# Paste the ixy info back into the file so we can plot the network of rivers
# with GMT (not this notebook). Reset index to row order
out = pd.concat([predict_ixy, sl_predict_output], axis='columns')
out.to_csv('RiverAtlas_GLORICH_'+branch+'_bias_adj_predictions.csv')

## Final steps and plotting

Next, we need to plot the data. Maps are made in GMT, see `fig07_predict_map.sh`.
Histograms are made in notebook `fig08_predict_hist.ipynb`.